In [100]:
# Importing necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import TomekLinks
from sklearn.utils.class_weight import compute_class_weight 

In [101]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import TomekLinks
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder 

In [102]:
# Load dataset (update the path to your dataset)
url= "D:\\Documents2\\python folder\\dataset\\weatherAUS.csv"
df= pd.read_csv(url) 

In [103]:
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [104]:
# Display basic information about the dataset
print(df.info())
df=df.dropna()
df['RainTomorrow']=LabelEncoder().fit_transform(df['RainTomorrow'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

In [105]:
# Assuming X_train has categorical columns like 'WindDir9am', 'WindDir3pm', etc.
categorical_columns = ['Location','WindGustDir', 'WindDir9am', 'WindDir3pm','RainToday'] 

for col in categorical_columns:
    label_enc = LabelEncoder()
    df[col] = label_enc.fit_transform(df[col])

In [106]:
# Split the dataset into features and target variable
X = df.drop(['RainTomorrow','Date'], axis=1)
y = df['RainTomorrow']

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 

In [107]:
from collections import Counter

In [108]:
X_train.head()

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday
140508,6,18.4,29.9,0.0,7.0,9.4,9,44.0,9,2,...,13.0,31.0,25.0,1015.0,1010.5,6.0,1.0,20.8,29.5,0
64503,9,17.3,34.8,0.0,8.0,13.0,3,41.0,3,11,...,17.0,28.0,19.0,1024.0,1020.9,0.0,1.0,28.0,34.2,0
75695,17,11.1,17.6,1.6,4.2,7.6,13,57.0,6,13,...,39.0,93.0,63.0,1000.7,1001.4,8.0,6.0,14.6,16.9,1
106889,25,14.5,33.3,0.0,10.2,9.2,4,54.0,1,3,...,33.0,24.0,5.0,1012.0,1005.7,5.0,1.0,22.3,32.8,0
87874,2,24.0,30.2,0.2,7.8,3.5,9,35.0,10,9,...,22.0,78.0,65.0,1013.9,1011.1,7.0,7.0,26.9,28.7,0


In [109]:
# Random Under Sampling
print("Original class distribution:", Counter(y_train))
rus = RandomUnderSampler(random_state=42)
X_rus, y_rus = rus.fit_resample(X_train, y_train) 
print("Original class distribution:", Counter(y_rus))

Original class distribution: Counter({0: 35194, 1: 9942})
Original class distribution: Counter({0: 9942, 1: 9942})


In [110]:
# Random Over Sampling
from imblearn.over_sampling import RandomOverSampler
print("Original class distribution:", Counter(y_train))
ros = RandomOverSampler(random_state=42)
X_ros, y_ros = ros.fit_resample(X_train, y_train) 
print("Original class distribution:", Counter(y_ros))

Original class distribution: Counter({0: 35194, 1: 9942})
Original class distribution: Counter({0: 35194, 1: 35194})


In [111]:
# SMOTE(Synthetic Minority Oversampling Techniqu)
print("Original class distribution:", Counter(y_train))
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X_train, y_train)
print("Original class distribution:", Counter(y_smote))

Original class distribution: Counter({0: 35194, 1: 9942})
Original class distribution: Counter({0: 35194, 1: 35194})


In [112]:
# Tomek Links
tl = TomekLinks()
X_tl, y_tl = tl.fit_resample(X_train, y_train) 

In [113]:
# Class Weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights)) 
class_weights

array([0.64124567, 2.2699658 ])

In [114]:
# Function to evaluate model performance
def evaluate_model(X_train, y_train, X_test, y_test, class_weights=None):
    model = RandomForestClassifier(class_weight=class_weights, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]
    
    print(classification_report(y_test, y_pred))
    print("AUC:", roc_auc_score(y_test, y_prob)) 

In [115]:
# Evaluate each sampling technique
print("Random Undersampling:")
evaluate_model(X_rus, y_rus, X_test, y_test)

print("\nRandom Oversampling:")
evaluate_model(X_ros, y_ros, X_test, y_test)

print("\nSMOTE:")
evaluate_model(X_smote, y_smote, X_test, y_test)

print("\nTomek Links:")
evaluate_model(X_tl, y_tl, X_test, y_test)

print("\nClass Weights:")
evaluate_model(X_train, y_train, X_test, y_test, class_weights_dict) 

Random Undersampling:
              precision    recall  f1-score   support

           0       0.94      0.81      0.87      8799
           1       0.54      0.81      0.65      2485

    accuracy                           0.81     11284
   macro avg       0.74      0.81      0.76     11284
weighted avg       0.85      0.81      0.82     11284

AUC: 0.8924888345872485

Random Oversampling:
              precision    recall  f1-score   support

           0       0.89      0.93      0.91      8799
           1       0.72      0.60      0.65      2485

    accuracy                           0.86     11284
   macro avg       0.80      0.77      0.78     11284
weighted avg       0.85      0.86      0.86     11284

AUC: 0.896745628904693

SMOTE:
              precision    recall  f1-score   support

           0       0.90      0.90      0.90      8799
           1       0.66      0.65      0.65      2485

    accuracy                           0.85     11284
   macro avg       0.78      